In [5]:
using CSV, DataFrames, HiGHS
# D = demand matrix (day of the week)
# n locations
# W = foot-traffic matrix (for each location, foot traffic for day of the week)

D = CSV.read("electricity/demand.csv", DataFrame; header=true)[:, 2]#7x1
mean_counts_df = CSV.read("ped_traffic/mean_counts.csv", DataFrame; header=true)
scenario_counts = CSV.read("ped_traffic/scenario_counts.csv", DataFrame; header=true)
W = mean_counts_df[:, 3:9]|> Matrix #110x7
location_ids = mean_counts_df[:, 1] # location IDs

mean_counts_df2 = CSV.read("ped_traffic/mean_counts_dist.csv", DataFrame; header=true)
W2 = mean_counts_df2[:, 3:9]|> Matrix #44x7
d = mean_counts_df2[:, 10] #44x1
location_ids2 = mean_counts_df2[:, 1] # location IDs

println(size(D))
println(size(W))
println(size(W2))
println(size(d))

(7,)
(110, 7)
(44, 7)
(44,)


In [6]:
D

7-element Vector{Float64}:
 1.1442044e8
 1.1817448863636364e8
 1.1911431638418078e8
 1.1900140677966101e8
 1.1614340571428572e8
 1.0544588439306359e8
 1.061848488372093e8

In [7]:
day_to_num = Dict(
    "Monday"    => 1,
    "Tuesday"   => 2,
    "Wednesday" => 3,
    "Thursday"  => 4,
    "Friday"    => 5,
    "Saturday"  => 6,
    "Sunday"    => 7
)

Dict{String, Int64} with 7 entries:
  "Friday"    => 5
  "Saturday"  => 6
  "Tuesday"   => 2
  "Thursday"  => 4
  "Wednesday" => 3
  "Monday"    => 1
  "Sunday"    => 7

In [44]:
using JuMP
using HiGHS
using LinearAlgebra

percent_demand = 0.001
steps_per_loc = 50
watts_converted = 0.1

max_tiles = 5

function optimize_installations(D::Vector{Float64})

    n, T = size(W)  # n locations, T = 7 days
    @assert length(D) == T "Demand vector D must have length 7"
    S = size(scenario_counts)[1] 

    model = Model(HiGHS.Optimizer)
    
    set_silent(model)

    @variable(model, x[1:n], Bin) # int decision vars
    @variable(model, y[1:S] >= 0) # int SCENARIO "decision" vars indicating fraction of demand not met

    @constraint(model, limit_tiles, sum(x[i] for i in 1:n) <= 5) # 
    @constraint(model, frac[s=1:S], y[s] <= 1)

    for s in 1:S # constraint each scenario
        counts = collect(scenario_counts[s,:])[2:n+1]
        t = day_to_num[scenario_counts[s,:day_of_week]]
        @constraint(model, sum(counts[i] * x[i] * watts_converted * steps_per_loc for i in 1:n) ≥ percent_demand * D[t] * (1-y[s]))
    end

    @objective(model, Min, sum(y[s] for s=1:S)/S)

    optimize!(model) # solve model

    status = termination_status(model)
    if status != MOI.OPTIMAL
        println("Warning: Solver returned status $status")
    end

    chosen_locations = findall(i -> value(x[i]) > 0.5, 1:n)
    min_num_locations = length(chosen_locations)

    return min_num_locations, chosen_locations, model, objective_value(model)
end

optimize_installations (generic function with 1 method)

In [45]:
optimize_installations(D)

(5, [9, 15, 19, 20, 78], A JuMP Model
├ solver: HiGHS
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 362
├ num_constraints: 867
│ ├ AffExpr in MOI.GreaterThan{Float64}: 252
│ ├ AffExpr in MOI.LessThan{Float64}: 253
│ ├ VariableRef in MOI.GreaterThan{Float64}: 252
│ └ VariableRef in MOI.ZeroOne: 110
└ Names registered in the model
  └ :frac, :limit_tiles, :x, :y, 0.2710861467529061)